In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

from bs4 import BeautifulSoup
import xml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Part I - Parsing of Wiki Data

#### Extraction and cleansing of the Wikipedia table.

In [8]:
Request=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Parser=BeautifulSoup(Request.text,"html.parser")

#### Processing, wrangling and filtering of the table

In [16]:

Table=Parser.table
Extraction=Table.find_all('tr')
Rows=len(Extraction)
Header=Extraction[0].text.split()
Datasource =[]
i=1 
while i < Rows :
    Postcode=Extraction[i].text.split('\n')[1]
    Borough=Extraction[i].text.split('\n')[2]
    Neighborhood=Extraction[i].text.split('\n')[3]
    Datasource.append((Postcode, Borough,Neighborhood))
    i=i+1

Q1df=pd.DataFrame(Datasource, columns=['PostalCode', 'Borough', 'Neighbourhood'])

Q1df['Borough'].replace('Not assigned', np.nan, inplace=True)
Q1df.dropna(subset=['Borough'], inplace=True)

Q1df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Q1df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

Q1df['Neighbourhood'].replace('Not assigned',"Queen's Park", inplace=True)
Q1df.head(20)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [17]:
Q1df.shape

(211, 3)

### Part II - Merge of data sources

#### Extraction of the geospatial data

In [22]:
df_geospatial=pd.read_csv('https://cocl.us/Geospatial_data')
df_geospatial.columns = ['PostalCode', 'Latitude', 'Longitude']
df_geospatial.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


#### Merge of datasources

In [30]:
postal_merge = pd.merge(df_geospatial,Q1df, on='PostalCode')
postal_merge.head(30)

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,Rouge
1,M1B,43.806686,-79.194353,Scarborough,Malvern
2,M1C,43.784535,-79.160497,Scarborough,Highland Creek
3,M1C,43.784535,-79.160497,Scarborough,Rouge Hill
4,M1C,43.784535,-79.160497,Scarborough,Port Union
5,M1E,43.763573,-79.188711,Scarborough,Guildwood
6,M1E,43.763573,-79.188711,Scarborough,Morningside
7,M1E,43.763573,-79.188711,Scarborough,West Hill
8,M1G,43.770992,-79.216917,Scarborough,Woburn
9,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [31]:
##Verification of merge
postal_merge.shape

(211, 5)

### Part III - Map

In [33]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [37]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(postal_merge['Latitude'], postal_merge['Longitude'], postal_merge['Borough'], postal_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto